In [1]:
# Import Libraries needed
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm
from os import listdir
from os.path import join
from collections import Counter
from scipy.io.wavfile import write
from datetime import datetime, timedelta

In [2]:
!mkdir data 

In [3]:
# Set file paths for mechanical readings csv and wav file location
DATA_PATH = "/kaggle/input/rain-data-enclosure-3-master/rainfall_sound_8k"
MECH_FILE_PATH = "/kaggle/input/rain-data-enclosure-3-master/rain_data_mechanical.csv"
NON_MECH_PATH = "/kaggle/input/rain-data-enclosure-3-master/rainfall_sound_8k"
Fs = 8000
MAX_LEN = 1368000 # 171 sec @ Fs 8k samples/sec
DAVIS_INTERVAL = 180

In [4]:
# Read mechanical raingauge readings as csv using pandas
mech_data = pd.read_csv(join(DATA_PATH, MECH_FILE_PATH))

# Convert time column to pandas time datatype
mech_data["time"]= pd.to_datetime(mech_data["time"])

# Get list of wav files and sort them in ascending order
wave_files = sorted(listdir(NON_MECH_PATH))

# Get total count of wav files available
N = len(wave_files)

In [5]:
mech_data.head(2)

,time,rainfall
0,2024-10-23 16:18:25.801606,0.0
1,2024-10-23 16:21:25.801604,1.2


In [6]:
# Function to parse date and time components from wav filenames
def filename_parser(filename):
    year, month, day, hour, minute, second, _ = map(int, filename.split(".")[0].split("_"))
    return datetime(year, month, day, hour, minute, second)

# Get the fist starting time available for wav files
start_time = filename_parser(wave_files[0])

# Get the last time available for wav files
end_time = filename_parser(wave_files[-1])

In [7]:
# Find overlapping time instances where mechanical readings and wav files are available
row_overlap = (mech_data["time"]>start_time)&(mech_data["time"]<end_time)
mech_data = mech_data[row_overlap]

In [8]:
mech_data["time_delta"] = mech_data["time"].diff()
mech_data["time_delta"] = mech_data["time_delta"].dt.total_seconds()

In [9]:
# To find the number of unique days where we have recording available
mech_data["time"] = mech_data["time"].astype('str')
mech_data[["YMD", "HMS"]] = mech_data["time"].str.split(" ",expand=True)
mech_data["YMD"].value_counts().keys()

Index(['2024-10-25', '2024-11-01', '2024-10-23', '2024-11-02', '2024-10-24',
       '2024-10-26', '2024-11-08', '2024-11-04', '2024-10-31', '2024-10-27',
       '2024-11-05', '2024-11-09'],
      dtype='object', name='YMD')

In [10]:
mech_data["time"]= pd.to_datetime(mech_data["time"])
mech_data = mech_data[["time", "rainfall"]]

In [11]:
# mech_data = mech_data[mech_data["rainfall"]!=0.0]

In [12]:
mech_data

,time,rainfall
0,2024-10-23 16:18:25.801606,0.0
1,2024-10-23 16:21:25.801604,1.2
2,2024-10-23 16:24:25.801603,0.6
3,2024-10-23 16:27:25.801605,4.2
4,2024-10-23 16:30:25.801606,0.6
...,...,...
548,2024-11-08 18:28:09.803826,0.4
549,2024-11-08 20:16:09.803825,0.0
550,2024-11-08 20:46:09.803827,0.0
551,2024-11-09 07:16:09.803827,0.6


In [13]:
# Function to load a wav file using librosa and get its sampling rate and duration
def load_wav(file_path, Fs=Fs):
    audio, Fs = librosa.load(file_path, sr=Fs)
    duration = librosa.get_duration(y=audio, sr=Fs)
    return audio, Fs, duration

In [14]:
mech_data

,time,rainfall
0,2024-10-23 16:18:25.801606,0.0
1,2024-10-23 16:21:25.801604,1.2
2,2024-10-23 16:24:25.801603,0.6
3,2024-10-23 16:27:25.801605,4.2
4,2024-10-23 16:30:25.801606,0.6
...,...,...
548,2024-11-08 18:28:09.803826,0.4
549,2024-11-08 20:16:09.803825,0.0
550,2024-11-08 20:46:09.803827,0.0
551,2024-11-09 07:16:09.803827,0.6


In [15]:
def get_fname_time(file_name):
    file_name_short = file_name.split("_")[:-1]
    year, month, day, hour, minute, second = map(int, file_name_short)
    fname_time = datetime(year, month, day, hour, minute, second)
    return fname_time

In [16]:
def ftime_to_fname(ftime):
    ftime_prev = ftime.strftime('%Y_%m_%d_%H_%M_%S')
    return ftime_prev

In [17]:
def filter_files(filenames, start_time, timestamp):
    filtered_files = []
    offset = timedelta(minutes=3)
    start_time -= offset
    timestamp += offset
    for filename in filenames:
        fname_time = get_fname_time(filename).replace(microsecond=0)
        if start_time <= fname_time <= timestamp:
            filtered_files.append(filename)
    return filtered_files

In [18]:
def combine_audio(timestamp, filenames):
    start_time = timestamp - timedelta(seconds=DAVIS_INTERVAL)
    combined_audio = []
    filtered_files = filter_files(filenames, start_time, timestamp)
    for filename in filtered_files:
        file_path = join(NON_MECH_PATH, filename)
        audio_data, sampling_rate, audio_duration = load_wav(file_path)
        fname_time = get_fname_time(filename).replace(microsecond=0)
        fname_end_time = fname_time+timedelta(seconds=audio_duration)
#         print(start_time, timestamp, fname_time, fname_end_time)
        if fname_time<start_time and fname_end_time<=start_time:
            continue
        elif fname_time<start_time and fname_end_time>start_time:
#             print("Getting parts of: ", filename)
            end_secs = (fname_end_time-start_time).total_seconds()
            init_secs = audio_duration-end_secs
            audio_data = audio_data[int(init_secs*sampling_rate):]
        elif fname_time==start_time or fname_end_time==timestamp:
#             print("Getting fullof")
            pass
        elif fname_time>start_time and fname_end_time<timestamp:
#             print("Getting fullof")
            pass
        elif fname_time<timestamp and fname_end_time>timestamp:
#             print("Getting parts of: ", filename)
            end_secs = (fname_end_time-timestamp).total_seconds()
            init_secs = audio_duration-end_secs
            audio_data = audio_data[:int(init_secs*sampling_rate)]
        elif fname_time>=timestamp:
            continue
        combined_audio.extend(audio_data)
    combined_audio = np.array(combined_audio)
    return combined_audio

In [19]:
data_basic = pd.DataFrame()
target = np.array([])
audio_lens=[]
for idx, row in tqdm(mech_data.iterrows(), total=mech_data.shape[0]):
    check_point = row["time"]
    rainfall = row["rainfall"]
    audio_sample = combine_audio(check_point, wave_files)
    audio_len = audio_sample.shape[0]
    audio_lens.append(audio_len)
    if audio_sample.shape[0]>=MAX_LEN:
        audio_sample = audio_sample[-MAX_LEN:]
#         write(join("data","audio_{}.wav".format(idx)), Fs, audio_sample)
        with open(join("data","audio_{}.npy".format(idx)), "wb") as f:
            np.save(f, audio_sample)
        data_row = {"checkpoint": check_point,
                    "fname": "audio_{}.wav".format(idx), 
                    "target": row["rainfall"]}
        data_basic = pd.concat([data_basic, pd.DataFrame([data_row])], ignore_index=True)

100%|██████████| 553/553 [04:27<00:00,  2.07it/s]


In [20]:
print(Counter(audio_lens))

Counter({1376000: 202, 1384000: 160, 1368000: 53, 1392000: 17, 1326412: 11, 1409588: 6, 1358412: 6, 1382412: 5, 1334412: 5, 1310412: 5, 1342412: 5, 1374412: 5, 1310430: 5, 1360000: 4, 1342430: 4, 1398412: 3, 1422412: 3, 1326430: 3, 1433588: 2, 1390412: 2, 1406412: 2, 1385588: 2, 1350412: 2, 1366412: 2, 1417588: 2, 1369588: 2, 1438412: 2, 1401588: 2, 1414412: 2, 1318430: 2, 1366430: 2, 1334430: 2, 1398430: 2, 1318412: 1, 1352000: 1, 1232000: 1, 1280000: 1, 1302412: 1, 1296000: 1, 1433570: 1, 1358430: 1, 1390430: 1, 1393570: 1, 1422430: 1, 1409570: 1, 1406430: 1, 1385570: 1, 1430430: 1, 1417570: 1, 1438430: 1, 1374430: 1, 1350430: 1, 964570: 1, 1294430: 1})


In [21]:
data_basic.to_csv("data_basic.csv")
mech_data.to_csv("mech_data.csv")

In [22]:
mech_data

,time,rainfall
0,2024-10-23 16:18:25.801606,0.0
1,2024-10-23 16:21:25.801604,1.2
2,2024-10-23 16:24:25.801603,0.6
3,2024-10-23 16:27:25.801605,4.2
4,2024-10-23 16:30:25.801606,0.6
...,...,...
548,2024-11-08 18:28:09.803826,0.4
549,2024-11-08 20:16:09.803825,0.0
550,2024-11-08 20:46:09.803827,0.0
551,2024-11-09 07:16:09.803827,0.6


In [23]:
data_basic

,checkpoint,fname,target
0,2024-10-23 16:18:25.801606,audio_0.wav,0.0
1,2024-10-23 16:21:25.801604,audio_1.wav,1.2
2,2024-10-23 16:24:25.801603,audio_2.wav,0.6
3,2024-10-23 16:27:25.801605,audio_3.wav,4.2
4,2024-10-23 16:30:25.801606,audio_4.wav,0.6
...,...,...,...
480,2024-11-08 14:13:09.803826,audio_541.wav,7.2
481,2024-11-08 14:16:09.803825,audio_542.wav,7.2
482,2024-11-08 14:19:09.803825,audio_543.wav,4.8
483,2024-11-08 14:22:09.803827,audio_544.wav,4.2
